In [7]:
import requests
import urllib.request

import re
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC

# 크롬 드라이버 초기화
def init_driver():
    return webdriver.Chrome("./chromedriver.exe")

In [38]:
driver = webdriver.Chrome('./chromedriver.exe')

In [ ]:
url = 'https://news.v.daum.net/v/20210403013444673'
driver = webdriver.Chrome('./chromedriver.exe')
driver.get(url)
v = driver.find_element_by_css_selector('div.cont_view')
v.text

In [ ]:
v = driver.find_element_by_css_selector('h2')
v.text

In [ ]:
html = urlopen(url)
bs = BeautifulSoup(html , 'lxml')
a = bs.select('h2#kakaoBody.screen_out')[0].text
print(a)

In [ ]:

## 기사 크롤링

url = 'https://news.v.daum.net/v/20210403013444673'


# 기사 Section 추출
# 이상하게 이 태그는 Selenium 으로 안 읽힌다.. 뭔데;
html = urlopen(url)
bs = BeautifulSoup(html , 'lxml')
article_type = bs.select('h2#kakaoBody.screen_out')[0].text

print(article_type)


In [ ]:

# 다음뉴스 기사 크롤링

article_data = [[]]

find_tag = ['h3.tit_view' , 'div.news_view p' , 
            'div.selectionbox.type-RECOMMEND.unselected' , 
          'div.selectionbox.type-LIKE.unselected' , 
           'div.selectionbox.type-IMPRESS.unselected', 
           'div.selectionbox.type-ANGRY.unselected' , 
           'div.selectionbox.type-SAD.unselected']


for i,tag in enumerate(find_tag):
    val = driver.find_elements_by_css_selector(tag)
    print(" ")
    
    for j in range(len(val)):
        print(val[j].text)

        

In [ ]:

# 다음뉴스 댓글 크롤링 (기사 하나)

# 웹 드라이버 실행

comment = driver.find_elements_by_css_selector('div.cmt_info')

# 댓글 아이디/날짜/텍스트/추천/비추천
Id = []
date = []
txt = []
recm = []

data = [ [] , [] , [] , [] , []]

find_tag = [  'a.link_nick' , 'span.txt_date', 'p.desc_txt.font_size_17' , 
            'button.btn_g.btn_recomm span.num_txt' ,
            'button.btn_g.btn_oppose span.num_txt'
           ]

for i,tag in enumerate(find_tag):

    val = driver.find_elements_by_css_selector('div.cmt_info ' + tag)

    for j in range(len(val)):
        data[i].append(val[j].text)
        
        

In [ ]:
idx_comment = 3

# 닉네임 / 댓글시간 / 댓글 내용 / 추천 / 비추천
data[0][idx_comment] , data[1][idx_comment] , data[2][idx_comment] , data[3][idx_comment] , data[4][idx_comment] 


In [ ]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC


# 댓글 더보기 누르기 , 이게 왜 되는건지는 모른다...
xpath = '//*[@id="alex-area"]/div/div/div/div[3]/div[3]/button'
driver.find_element_by_xpath(xpath).send_keys(Keys.RETURN)

# 클린봇 설정 누르기
xpath = '//*[@id="alex-area"]/div/div/div/div[3]/div[1]/button'
driver.find_element_by_xpath(xpath).send_keys(Keys.RETURN)

# 클린봇 설정 바꾸기
xpath = '//*[@id="alex-area"]/div/div/div[2]/div[2]/div/div[2]/dl/dd/button'
driver.find_element_by_xpath(xpath).send_keys(Keys.RETURN)

# 클린봇 설정 창 닫기
xpath = '//*[@id="alex-area"]/div/div/div[2]/div[2]/div/a'
driver.find_element_by_xpath(xpath).send_keys(Keys.RETURN)

# 댓글 설정 바꾸기 
# 안 바꾸면추천댓글을 자동으로 보게됨 (최대 15개)
xpath = '//*[@id="alex-area"]/div/div/div/div[3]/ul[1]/li[3]/button'
driver.find_element_by_xpath(xpath).send_keys(Keys.RETURN)
    

    

In [11]:

## 한 기사에 대해서 댓글들 크롤링하기

url = 'https://news.v.daum.net/v/20210403184201857'

def crawling(driver , url , clean_bot = False , all_reply = False , nested_reply = False):
    

    driver.get(url)
    driver.implicitly_wait(3)
    
    # 그냥들어가면 추천댓글을 자동으로 보게됨 (대충 최대 15개 정도)
    if all_reply:
        xpath = '//*[@id="alex-area"]/div/div/div/div[3]/ul[1]/li[3]/button'
        driver.find_element_by_xpath(xpath).send_keys(Keys.RETURN)
    
    # 클린봇 이용
    if clean_bot :
        xpath = '//*[@id="alex-area"]/div/div/div/div[3]/div[1]/button'
        driver.find_element_by_xpath(xpath).send_keys(Keys.RETURN)
        xpath = '//*[@id="alex-area"]/div/div/div[2]/div[2]/div/div[2]/dl/dd/button'
        driver.find_element_by_xpath(xpath).send_keys(Keys.RETURN)
        xpath = '//*[@id="alex-area"]/div/div/div[2]/div[2]/div/a'
        driver.find_element_by_xpath(xpath).send_keys(Keys.RETURN)

    # 댓글 더보기 펼치기
    xpath = '//*[@id="alex-area"]/div/div/div/div[3]/div[3]/button'
    while(1):
        try: driver.find_element_by_xpath(xpath).send_keys(Keys.RETURN)
        except: break
      
    
    # 대댓글 보기
    if nested_reply:
        xpath = '//*[@id="comment652773577"]/div/div/span[1]/button'
        while(1):
            try: driver.find_element_by_xpath(xpath).send_keys(Keys.RETURN)
            except: break
                
  
    data = []
    find_tag = ['p.desc_txt.font_size_17' ]
    
#     data = [ [] , [] , [] , [] , []]

#     find_tag = [  'a.link_nick' , 'span.txt_date', 'p.desc_txt.font_size_17' , 
#                 'button.btn_g.btn_recomm span.num_txt' ,
#                 'button.btn_g.btn_oppose span.num_txt'
#                ]

    for i,tag in enumerate(find_tag):

        val = driver.find_elements_by_css_selector('div.cmt_info ' + tag)

        for j in range(len(val)):
            #data[i].append(val[j].text)
            data.append(val[j].text)
            
            
    return data



In [14]:

%%time

# 댓글 크롤링 테스트 1  :  클린봇 False , 전체 댓글 False , 대댓글 False 
# 댓글 16개 , 10.2초

url = 'https://news.v.daum.net/v/20210403110603463'

# 대댓글 말고 댓글만 수집
a = crawling(driver , url , False , False , False)

print(len(a))


16
Wall time: 10.2 s


In [13]:

%%time

# 댓글 크롤링 테스트 2  :  클린봇 False , 전체 댓글 True , 대댓글 False 
# 댓
# 댓글이 많은 기사는 "더보기" 펼치는데만 한세월임....

url = 'https://news.v.daum.net/v/20210403110603463'

# 대댓글 말고 댓글만 수집
a = crawling(driver , url , False , True , False)

print(len(a))


KeyboardInterrupt: 

In [15]:

%%time

# 댓글 크롤링 테스트 3  :  클린봇 False , 전체 댓글 False , 대댓글 True 
# 댓글 16개 , 5.54초

url = 'https://news.v.daum.net/v/20210403110603463'

# 대댓글 말고 댓글만 수집
a = crawling(driver , url , False , False , True)

print(len(a))


3
Wall time: 23.6 s


In [ ]:

%%time

# 댓글 크롤링 테스트 3  :  클린봇 False , 전체 댓글 True , 대댓글 False 
# 댓글 16개 , 5.54초

url = 'https://news.v.daum.net/v/20210403110603463'

# 대댓글 말고 댓글만 수집
a = crawling(driver , url , False , True , False)

print(len(a))


In [ ]:

## 기사 url 따기

url = 'https://news.daum.net/breakingnews/society?page=1'
html = urlopen(url)
bs = BeautifulSoup(html , 'lxml')
a = bs.select('div.box_etc a.link_thumb')
print(a)

In [ ]:
%%time

## 특정 날짜에 대해서 "사회" 에서 각 기사 링크 따오기

# 시간 좀 걸림.. , 유효하지 않은 날짜는 오늘 날짜로 이동해짐

society_url = "https://news.daum.net/breakingnews/society"

page = 0
day_urls = []
    
date = "20210321"
last_item = ""

while(1):
    page += 1
    page_url = society_url + "?page=" + str(page) + '&regDate=' + date
    
    
    if page%50 == 0:
        print(page_url)
    html = urlopen(page_url)
    bs = BeautifulSoup(html , 'lxml')

    a = bs.select('div.box_etc a.link_thumb')

    for c in a:
        day_urls.append(c.get('href'))
    
    if last_item == c.get('href'): break
    else:
        last_item = c.get('href')
    
        

print( date , " 전체 기사 수 : " , len(day_urls))



In [17]:

## 시작날짜 ~ 끝날짜 를 받아서 중간에 있는 날짜 골라내기

# timedate , zfill 이용

from datetime import date
from datetime import timedelta

def return_dates(start_date , end_date):

    days = [start_date]
    Date = date(int(start_date[:4]), int(start_date[4:6]), int(start_date[6:]))
    end_date = date(int(end_date[:4]), int(end_date[4:6]), int(end_date[6:]))

    delta = timedelta(days=1)

    while(1):
        Date += delta
        str_date = str(Date.year) + str(Date.month).zfill(2) + str(Date.day).zfill(2)
        days.append(str_date)
        if Date == end_date: break
            
    return days
   
    




In [ ]:
start_date = '20210320'
end_date = "20210401"

DATES = return_dates(start_date , end_date)
DATES

In [19]:
%%time

## 특정 섹터 , 원하는 기간을 통해 기사 url 따기

# 다음 뉴스의 섹션은 6개로 구성
# society , politics , economic , foreign , culture , digital

# 원하는 범위 지정
Section = "culture"
start_date = "20210327"
end_date = "20210328"

DATES = return_dates(start_date , end_date)
print("크롤링할 전체 날짜 : " , len(DATES))

Total_URLS = []

for day in DATES:
    print("현재 크롤링 날짜 : " , day)
    url = "https://news.daum.net/breakingnews/" + Section

    page = 0
    num = 0
    last_item = ""

    while(1):
        page += 1
        page_url = url + "?page=" + str(page) + '&regDate=' + day
        html = urlopen(page_url)
        bs = BeautifulSoup(html , 'lxml')
        a = bs.select('div.box_etc a.link_thumb')
        for c in a:
            Total_URLS.append(c.get('href'))
            num += 1

        if last_item == c.get('href'): break
        else:
            last_item = c.get('href')

    print(Section + " 영역 " + day + "기사 수 : " + str(num))
print( " 크롤링할 전체 기사 수 : " , len(Total_URLS))
    



크롤링할 전체 날짜 :  2
현재 크롤링 날짜 :  20210327
culture 영역 20210327기사 수 : 457
현재 크롤링 날짜 :  20210328
culture 영역 20210328기사 수 : 448
 크롤링할 전체 기사 수 :  905
Wall time: 22.9 s


In [36]:
%%time

## 특정 섹터 , 원하는 기간을 통해 기사 url 따기

# 다음 뉴스의 섹션은 6개로 구성
# society , politics , economic , foreign , culture , digital

# 원하는 범위 지정
category = "ranking/popular/"
start_date = "20210401"
end_date = "20210403"

DATES = return_dates(start_date , end_date)
print("크롤링할 전체 날짜 : " , len(DATES))

Total_URLS = []

for day in DATES:
    print("현재 크롤링 날짜 : " , day)
    url = "https://news.daum.net/" + category

    page = 0
    num = 0
    last_item = ""

    while(1):
        page += 1
        page_url = url  + '?regDate=' + day
        print(page_url)
        html = urlopen(page_url)
        bs = BeautifulSoup(html , 'lxml')
        a = bs.select('div.cont_thumb a.link_txt')
        for c in a:
            Total_URLS.append(c.get('href'))
            num += 1

        if last_item == c.get('href'): break
        else:
            last_item = c.get('href')

    print(Section + " 영역 " + day + "기사 수 : " + str(num))
print( " 크롤링할 전체 기사 수 : " , len(Total_URLS))
    



크롤링할 전체 날짜 :  3
현재 크롤링 날짜 :  20210401
https://news.daum.net/ranking/popular/?regDate=20210401
https://news.daum.net/ranking/popular/?regDate=20210401
culture 영역 20210401기사 수 : 108
현재 크롤링 날짜 :  20210402
https://news.daum.net/ranking/popular/?regDate=20210402
https://news.daum.net/ranking/popular/?regDate=20210402
culture 영역 20210402기사 수 : 108
현재 크롤링 날짜 :  20210403
https://news.daum.net/ranking/popular/?regDate=20210403
https://news.daum.net/ranking/popular/?regDate=20210403
culture 영역 20210403기사 수 : 108
 크롤링할 전체 기사 수 :  324
Wall time: 1.83 s


In [20]:
Total_URLS[:30]

['https://v.daum.net/v/20210327235601825',
 'https://v.daum.net/v/20210327230726609',
 'https://v.daum.net/v/20210327230003563',
 'https://v.daum.net/v/20210327222701365',
 'https://v.daum.net/v/20210327222039307',
 'https://v.daum.net/v/20210327221944301',
 'https://v.daum.net/v/20210327221515279',
 'https://v.daum.net/v/20210327220536222',
 'https://v.daum.net/v/20210327220132196',
 'https://v.daum.net/v/20210327220123194',
 'https://v.daum.net/v/20210327220116192',
 'https://v.daum.net/v/20210327220052191',
 'https://v.daum.net/v/20210327220032188',
 'https://v.daum.net/v/20210327220019184',
 'https://v.daum.net/v/20210327215907169',
 'https://v.daum.net/v/20210327215459131',
 'https://v.daum.net/v/20210327214606048',
 'https://v.daum.net/v/20210327214537035',
 'https://v.daum.net/v/20210327214351016',
 'https://v.daum.net/v/20210327213932971',
 'https://v.daum.net/v/20210327213842959',
 'https://v.daum.net/v/20210327213809951',
 'https://v.daum.net/v/20210327213704938',
 'https://v

In [47]:
%%time
#data = []

driver.implicitly_wait(3)


for url in Total_URLS[50:]:
    data.append(crawling(driver , url))

print(len(data))



InvalidArgumentException: Message: invalid argument
  (Session info: chrome=89.0.4389.114)


In [48]:
len(data)

76

In [50]:
ls

 C 드라이브의 볼륨에는 이름이 없습니다.
 볼륨 일련 번호: 8E1E-1CEA

 C:\Users\Ando\Developer\WebCrawler\study\Coupang_Scrapping 디렉터리

2021-04-04  오후 06:56    <DIR>          .
2021-04-04  오후 06:56    <DIR>          ..
2021-04-03  오후 10:27    <DIR>          .ipynb_checkpoints
2021-04-04  오후 06:56            62,378 Article WebCrawler.ipynb
2021-01-27  오전 04:53        10,695,680 chromedriver.exe
2021-01-26  오전 03:36            34,305 Coupang 우유 Searching Result.xlsx
2021-04-03  오후 05:55           635,540 Coupang.ipynb
2021-01-27  오전 11:59            16,471 Coupang.md
2021-01-26  오후 02:33    <DIR>          img
2021-04-04  오후 06:54           137,767 trash text.csv
2021-04-03  오후 10:31             2,921 Untitled.ipynb
               7개 파일          11,585,062 바이트
               4개 디렉터리  14,349,828,096 바이트 남음


In [27]:
data[:50]

[['ㅎ여러분  생각을  댓글로  남겨주세요 보낸이  에게도  보내겠습니다...JTBC 드라마 설** 촬영중지 해야 합니다\n전뭐가 잘못했다고   생각이  안듭니다 \n청원에 동의해주세요 https://www1.president.go.kr/petitions/about\n설레발 치지마라...하는 짓이 "토착 왜구" 같노...\n아님 말고...? ㅎㅎㅋㅋㅋ\n 어지간히 ㅂㄷㅂㄷ 거리네... ㅎㅎㅋㅋ\n그니깐 당신 얘기는 카더라 통신이나',
  '안기부 미화할수밖에 없는 이유\n작가 남편이  검사출신이고\n신의저울 집필시 남편친구 윤석열 한테 도움을 받았다함\n자신이 모르는 분야의 전문가라고 생각할테고\n당연히 그들의 말이  정의감으로 느껴질듯',
  '작가 피디 제작사 영구 퇴출!!!!\n\n중국돈으로 블핑 한류스타 내세워서 \n대한민국의 민주주의 가치와 역사성을 훼손하려는 \n자들은 영원히 퇴출시켜야함\n\n조선구마사는 아직도 해외에 송출되고 있다고 함\n\n설강화는 아예 세상에 나오지 못하게 해야함 \n\n배우들도 매국노짓 그만두고 빠져라',
  '대쪽같은 안기부?\n그럼 이근안은 직업정신이 투철한 고문예술가인가요?\n블랙코미디고 로맨스고,\n어디 버무릴만한 배경을 들고올 것이지,\n지금이 미얀마 사태를 두고도 \n풍자하고 사랑하고 그런 드라마가 가능하겠냐고요!'],
 [],
 [],
 [],
 ['텔레비전 보니..역시 돈있는 집안은 다르네.\n코로나로 일반국민들은 장례식도 제대로 고인에 대한 예를 못하는 경우가 많은데 신회장 장례식 영상보니 방역수칙은 재벌들에게는 딴 나라 이야기이네.',
  '얼마전 백기완 선생  돌아가셨을때 이렇게 했어봐라!\n자본과 판검새권력과 수구부패야당세력하고만 손잡고 왜곡보도에만 열 올리지말고 언론들아!',
  '글로벌 음식이된..\n맛있는 라면과 새우깡 만들어 주셔서 감사드립니다.   삼가고인의명복을빕니다.',
  '삼양 정치로 엿먹임.\n누굴까 .. 3놈인데',
  '참 이렇게 미화되네. 김기춘 그 늙은이와 짜고 친 고스톱을 알

In [49]:
import pandas as pd

df = pd.DataFrame({'text' : data})

df.to_csv("trash text.csv")

In [54]:

comment_data = []

for sentences in data:
    for sentence in sentences:
        comment_data.append(sentence)
        
len(comment_data)
    
    

1040

In [55]:

df = pd.DataFrame({'text' : comment_data})

df.to_csv("trash text.csv")